# GUI Design

## Set backend, import libraries and hardware overlay

Converts the backend to the jupyter notebook bersion of matplotlib:

In [ ]:
%matplotlib notebook

Imports required libraies for notebook

In [ ]:
import numpy as np  #used to convert image to numpy array, required for hardware acceleration
import ipywidgets as widgets  #used for GUI widgets
from ipywidgets import HBox, VBox  #Allows for widgets to be stored in boxes, allinged hoirzontally and vertically
from IPython.display import display 
from PIL import Image  #used for image processing
import _thread  #Uses multithreading so slider widgets can be used with other code simultaneously
import matplotlib.pyplot as plt  #matplotlib is used for interactive image and click detection
import matplotlib.image as mpimg
import matplotlib as mpl
from matplotlib.widgets import RectangleSelector
from pynq import Overlay
from pynq import Xlnk

imports hardware overlay

In [ ]:
overlay = Overlay("bitstream/design_1_wrapper.bit")
#overlay?

In [ ]:
dma = overlay.axi_dma

Create DMA arrays and instantiate xlink:

In [ ]:
IMG_SIZE = (320,240) 

# Make an Xlnk instance to control memory allocation
xlnk = Xlnk()

# Input array for DMA use
in_dma_array  = xlnk.cma_array(shape=(IMG_SIZE[1]+2, IMG_SIZE[0]+2, 4), dtype=np.uint8)

# Output array for DMA use
out_dma_array = xlnk.cma_array(shape=(IMG_SIZE[1],IMG_SIZE[0],4), dtype=np.uint8)

Define filter function implemented in hardware:

In [ ]:
def effect(image_array, BlurOrSharpen = 120, reference = 127, BlurSelect = 130, LUTSelect = 1, LUTIntensity = 0):
    
    #Set system parameters
    overlay.fullsystembuild_0.write(0x00, LUTSelect)    #LUT Selection
    overlay.fullsystembuild_0.write(0x04, reference)  #reference  
    overlay.fullsystembuild_0.write(0x08, BlurSelect)  #Blur/Sharp Request
    overlay.fullsystembuild_0.write(0x0c, BlurOrSharpen)  #Blur or Sharp
    overlay.fullsystembuild_0.write(0x10, LUTIntensity)    #LUT Intensity
    
    # Copy image array into dma buffer
    in_dma_array[:, :, :3]= image_array[:, :, :3]
        
    # Perform DMA transfers in both directions
    dma.recvchannel.transfer(out_dma_array)
    dma.sendchannel.transfer(in_dma_array)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    # Return processed image array from PL 
    return out_dma_array

## Define functions used in notebook for image processing

Defines functions used for iamge processing:

In [ ]:
#import image from file address
def get_Image(path):
    imported_image = Image.open(path)   #find image from given path
    imported_image.load()   #decode image to python file format
    image_array = np.array(imported_image)  #converts image to numpy array
    return imported_image, image_array   #return image and array as tuple

def find_image(path):
    imported_image = Image.open(path)   #find image from given path
    imported_image.load()   #decode image to python file format
    image_array = np.array(imported_image)  #converts image to numpy array
    return imported_image   #return image and array as tuple

#resize image according to given scale factor
def scale_Sticker(image,scalefactor):
    width, height = image.size  #find original image size
    resized_width, resized_height=int(width*scalefactor), int(height*scalefactor) #find new diemensions by multipying with scale factor
    resized_image = image.resize((resized_width, resized_height)) #resize image
    return resized_image

#rotate image by given angle (degrees)
def rotate_Sticker(image, angle):
    im = image.convert('RGBA')  #convert sticker to RGBA image format for transparent mask
    rot_im = im.rotate( angle, expand=1 )  #rotate image, keeping all pixels
    return rot_im

def reapply_stickers():
    global base_image
    base_image = filtered_image

    if len(store_type)>0:
        for i in range(len(store_type)):
            stickk = sticker_list[store_type[i]]
            stickk = scale_Sticker(stickk,store_scale[i])
            stickk = rotate_Sticker(stickk,store_rotation[i])
            stickk = stickk.convert('RGBA')  #RGBA is used to allow sticker to have transparent background for rotating
            base_image.paste(stickk, store_offset[i], stickk) #add sticker to image, using sticker transparency mask
            
    plt.imshow(base_image)  #display final image
    
def apply_filter():
    global filtered_image
    filtered_image = original_image
    filter_array = np.array(filtered_image, dtype=np.uint8)
    red_images = filter_array[:,:,0]
    blue_images = filter_array[:,:,2]
    green_images = filter_array[:,:,1]
    
    rbg = np.dstack((green_images, blue_images, red_images))  # stacks 4 h x w arrays -> h x w x 4
    image_output = effect(rbg,select_blursharpen,127,select_blursharpenreq,select_lut,select_intensity)
    
    red_output = image_output[:,:,2]
    blue_output = image_output[:,:,1]
    green_output = image_output[:,:,0]
    rgb = np.dstack((red_output, green_output, blue_output))
    filtered_image=Image.fromarray(rgb[:,:,:3], mode='RGB')


Define functinos for interactive GUI:

In [ ]:
#Handle the matplotlib image being clicked, adds sticker and displays new image
def onclick(event):
    global remove
    cx, cy = event.xdata, event.ydata  #find click co-ordinates
    if remove == 0:
        sticker_width, sticker_height = updated_sticker.size  #find sticker size
        offset = (int(cx - (sticker_width // 2)), int(cy - (sticker_height // 2)))  #finds co-ords of top left corner of the sticker, assuming click is centre
        store_scale.append(sscale)
        store_rotation.append(srotation)
        store_offset.append(offset)
        store_type.append(stype)
        stick = updated_sticker.convert('RGBA')  #RGBA is used to allow sticker to have transparent background for rotating
        base_image.paste(stick, offset, stick) #add sticker to image, using sticker transparency mask
        plt.imshow(base_image)  #display final image

    elif (len(store_type) > 0):  #if stickers have been added 
        global found
        #find last sticker placed in area to remove
        found = 0
        for i in range(len(store_type)):
            test_stick = rotate_Sticker(sticker_list[store_type[i]],store_rotation[i])       
            sticker_width, sticker_height = test_stick.size  #find sticker size of original sticker
            minx_lim = store_offset[i][0]
            miny_lim = store_offset[i][1]
            maxx_lim = store_offset[i][0]+int((sticker_width *store_scale[i]))
            maxy_lim = store_offset[i][1]+(sticker_height * store_scale[i])

            if cx >= minx_lim and cx <= maxx_lim and cy >= miny_lim and cy <= maxy_lim:  #check if click was on sticker
                    found = 1
                    clicked_stick_index = i  #finds click sticker, if multiple clicked takes most recently placed

        if found == 1:
            del store_type[clicked_stick_index]
            del store_offset[clicked_stick_index]
            del store_rotation[clicked_stick_index]
            del store_scale[clicked_stick_index]
            reapply_stickers()  #reapply all other sticker
            
            
    #Scale and rotate sticker on slider update
def set_Scale(scalefactor, angle): 
    global updated_sticker
    global sscale, srotation
    global angles
    angles = angle
    sscale = scalefactor
    srotation = angle
    updated_sticker = sticker  #takes original sticker as opposed to previously scaled/rotated sticker
    updated_sticker = scale_Sticker(updated_sticker,scalefactor)
    updated_sticker = rotate_Sticker(updated_sticker,angle)


#define slider operation
def slider_Scale():  #function to allow user interaction with sliders
    try:
        set_Scale(1,0)
        widgets.interact(set_Scale, scalefactor =scaleValue_Slider, angle = rotation_Slider)
    except Exception:
        import traceback
        print(traceback.format_exc())  #display error message if unexpected error occurs

        
    
#define filter buttons click handlers    

#removes filters
def filter_original_onclick(b):
    global filtered_image
    filtered_image = original_image  #resets to unchanged image
    base_image = filtered_image  #sets displayed image to unchanged image
    reapply_stickers()  #reapplies users stickers


# applies negative filter when clicked
def filter_negative_onclick(b):
    global select_lut  #select_lut is change to alter filter type
    base_image = original_image  #filter is applied to original image, not reapplied over existing filters
    select_lut = 0  #lut for negative filter
    apply_filter()  #applies filter
    reapply_stickers()  #reapplies stickers to image


# applies warm filter when clicked
def filter_warm_onclick(b):
    global select_lut
    select_lut = 1
    apply_filter()
    reapply_stickers()

# applies cool filter when clicked
def filter_cool_onclick(b):
    global select_lut
    select_lut = 2
    apply_filter()
    reapply_stickers()
    
# apples bright clicked
def filter_bright_onclick(b):
    global select_lut
    select_lut = 3
    apply_filter()
    reapply_stickers()

# applies dim filter when clicked
def filter_dim_onclick(b):
    global select_lut
    select_lut = 4
    apply_filter()
    reapply_stickers()
    
#applies blur filter when clicked
def filter_blur_onclick(b):
    global select_blursharpenreq
    global select_blursharpen
    select_blursharpenreq = 128
    select_blursharpen = 126
    apply_filter()
    reapply_stickers()
    
# no blur or sharpen
def filter_default_onclick(b):
    global select_blursharpenreq
    global select_lut
    select_lut=5
    select_blursharpenreq =126
    apply_filter()
    reapply_stickers()
   
 # apply sharpen
def filter_sharp_onclick(b):
    global select_blursharpenreq
    global select_blursharpen
    select_blursharpenreq = 128
    select_blursharpen = 128
    apply_filter()
    reapply_stickers()
 
 # min filter
def filter_none_onclick(b):
    global select_intensity
    select_intensity = 0
    apply_filter()
    reapply_stickers()
    
    
def filter_low_onclick(b):
    global select_intensity
    base_image = original_image
    select_intensity = 1
    apply_filter()
    reapply_stickers()
    
def filter_medium_onclick(b):
    global select_intensity
    select_intensity = 2
    apply_filter()
    reapply_stickers()
    
def filter_high_onclick(b):
    global select_intensity
    select_intensity = 3
    apply_filter()
    reapply_stickers()
    
def filter_max_onclick(b):
    global select_intensity
    select_intensity = 4
    apply_filter()
    reapply_stickers()
        
    
#click handler for first sticker
def sticker0_onclick(b):
    global sticker, updated_sticker
    global stype
    global remove
    remove = 0  #sticker will be added not removed when image clicked
    stype = 0  #store sticker type in list for reapplying stickeres
    sticker = sticker_list[0]  #sets active sticker from list containing all sticker choices
    updated_sticker = sticker  #sets active sticker
    
def sticker1_onclick(b):
    global sticker, updated_sticker
    global stype
    global remove
    remove = 0
    stype = 1
    sticker = sticker_list[1]
    updated_sticker = sticker
    
    
def sticker2_onclick(b):
    global sticker, updated_sticker
    global stype
    global remove
    remove = 0
    stype = 2
    sticker = sticker_list[2]
    updated_sticker = sticker
    
def sticker3_onclick(b):
    global sticker, updated_sticker
    global stype
    global remove
    global remove_filter
    remove_filter=0
    remove = 0
    stype = 3
    sticker = sticker_list[3]
    updated_sticker = sticker
    
def remove_onclick(b):
    global original_image,base_image, filtered_image, crop_sample
    
    if cropped == 0:
        original_image = find_image(base_address) #reset to original image
    else:
        original_image = find_image('assets/crop.png')
    
    original_image = original_image.resize((322,242))
    base_image = original_image
    filtered_image = base_image
    #global select_lut, select_blursharpenreq  #applys no filter
    #select_blursharpenreq=126
    #select_lut = 5
    global store_scale,store_rotation,store_offset,store_type #clear sticker list
    store_scale.clear()
    store_rotation.clear()
    store_offset.clear()
    store_type.clear()
    apply_filter()
    reapply_stickers()
    #plt.imshow(base_image)  #display final image  
    
def remove_filter_onclick(b):
    global base_image
    global filtered_image
    global select_lut, select_blursharpenreq  #applys no filter
    select_blursharpenreq=126
    select_lut = 5
    filtered_image = original_image
    base_image = filtered_image
    reapply_stickers()
    
    
def mirror_onclick(b):
    global sample_image
    sample_image = sample_image.transpose(Image.FLIP_LEFT_RIGHT)  #mirrors image
    plt.imshow(sample_image)  #display mirror image

    
def crop_onclick(b):
    global sample_image
    global cropped
    global crop_sample
    global base_image, original_image, filtered_image
    crop_area = (x1,y1,x2,y2)  #finds highlighted area
    crop_sample = sample_image.crop(crop_area)  #crops image
    crop_sample = crop_sample.resize((322,242))  #resize image so it can be filtered
    base_image = crop_sample
    original_image = base_image
    filtered_image=base_image
    crop_sample.save("assets/crop.png")
    cropped = 1  #flag crop, crop image will be used in editor instead of original

    
def save_onclick(b):
    global saved_image, saved
    saved_image=base_image
    saved_image=rotate_Sticker(saved_image,angles)
    saved_image.save("assets/output.png")
    saved = 1
    

In [ ]:
#set global variable for filtering
global select_blursharpen, select_blursharpenreq, select_LUT, select_intensity
select_blursharpen =126
select_blursharpenreq=127
select_LUT =5
select_intensity = 5

global cropped,angles,saved
angles = 0
saved = 0
cropped = 0

#imports and stores stickers
global sticker_list
global applied_stickers
global remove, stype
global base_address
stype = 0
remove = 0
stickers = ["assets/koalaStickerOriginalSize.png","assets/noods.png","assets/sacracasm.png","assets/care.png"]
sticker_list = []

#store information on stickers already placed by user
store_scale=[]
store_rotation=[]
store_offset=[]
store_type=[]

#stores all possible stickers in a list
for i in range(len(stickers)):
    sadd = find_image(stickers[i])
    
    if i != 3:
        sadd=scale_Sticker(sadd,0.1)
    
    else:
        sadd=scale_Sticker(sadd,0.25)
    
    sticker_list.append(sadd)
    
base_address = "assets/peppers.png"  #stores address of base image
sticker_address =stickers[0]  #stores address of active sticker

In [ ]:
global original_image, base_image, sticker, filtered_image

#Get and display base image
base_image, base_image_array = get_Image(base_address)
base_image = base_image.resize(IMG_SIZE).convert('RGB')
image_array = np.array(base_image, dtype=np.uint8)
base_image_arr = np.pad(image_array, ((1,1),(1,1),(0,0)), 'symmetric')

base_image=Image.fromarray(base_image_arr[:,:,:3], mode='RGB')


original_image = base_image
filtered_image=base_image

#find and print base image size
base_width, base_height = base_image.size
print("The base image size is {}x{} pixels".format(base_width,base_height))

#Get and display sticker
sticker, sticker_array = get_Image(sticker_address)
#display(sticker)

#resize sticker
sticker = scale_Sticker(sticker,0.1)
updated_sticker = sticker
sticker_width, sticker_height = sticker.size

#find and print sticker size
sticker_width, sticker_height = sticker.size
print("The Sticker size is {}x{} pixels".format(sticker_width,sticker_height))


In [ ]:
#Creates buttons to apply filters and specifies parameters 
#Button to apply first filter to base image
#Button to apply second filter to base image
filter_negative= widgets.Button(
    description='Negative Filter',
    disabled=False,
    button_style='info', 
    icon='tint' 
)

#Button to apply third filter to base image
filter_warm= widgets.Button(
    description='Warm Filter',
    disabled=False,
    button_style='primary',
    icon='tint'
)

filter_cool= widgets.Button(
    description='Cool Filter',
    disabled=False,
    button_style='info',
    icon='tint'
)

filter_bright= widgets.Button(
    description='Bright Filter',
    disabled=False,
    button_style='primary',
    icon='tint'
)

filter_dim= widgets.Button(
    description='Dim Filter',
    disabled=False,
    button_style='info',
    icon='tint'
)

filter_blur= widgets.Button(
    description='Blur',
    disabled=False,
    button_style='info',
    icon='tint'
)

filter_default= widgets.Button(
    description='Remove blur/sharpen',
    disabled=False,
    button_style='primary',
    icon='tint'
)

filter_sharp= widgets.Button(
    description='Sharp',
    disabled=False,
    button_style='info',
    icon='tint'
)


undo_button= widgets.Button(
    description='Remove Stickers',
    disabled=False,
    button_style='warning',
    icon='undo' 
)

remove_filter_button= widgets.Button(
    description='Remove Filters',
    disabled=False,
    button_style='warning',
    icon='undo' 
)

crop_button= widgets.Button(
    description='Crop sticker',
    disabled=False,
    button_style='warning',
    icon='undo' 
)

save_button= widgets.Button(
    description='Save',
    disabled=False,
    button_style='warning',
    icon='life-saver' 
)


none_button= widgets.Button(
    description='Min',
    disabled=False,
    button_style='primary',
    icon='battery-0' 
)

low_button= widgets.Button(
    description='Low',
    disabled=False,
    button_style='info',
    icon='battery-1' 
)

medium_button= widgets.Button(
    description='Medium',
    disabled=False,
    button_style='primary',
    icon='battery-2' 
)

high_button= widgets.Button(
    description='High',
    disabled=False,
    button_style='info',
    icon='battery-3' 
)

max_button= widgets.Button(
    description='Max',
    disabled=False,
    button_style='primary',
    icon='battery-4' 
)
ui_buttons = HBox([remove_filter_button,undo_button,save_button])
filter_buttons = HBox([filter_negative, filter_warm, filter_cool, filter_bright, filter_dim])  #list containing buttons for filtering
intensity_buttons=HBox([none_button,low_button,medium_button,high_button,max_button])
blur_buttons= HBox([filter_blur,filter_default,filter_sharp])

filter_ui = VBox([ui_buttons,filter_buttons,intensity_buttons,blur_buttons])  #create horizontal box containing buttons 


#Add click handlers to each button
filter_negative.on_click(filter_negative_onclick)
filter_warm.on_click(filter_warm_onclick)
filter_cool.on_click(filter_cool_onclick)
filter_bright.on_click(filter_bright_onclick)
filter_dim.on_click(filter_dim_onclick)
undo_button.on_click(remove_onclick)
filter_blur.on_click(filter_blur_onclick)
filter_default.on_click(filter_default_onclick)
filter_sharp.on_click(filter_sharp_onclick)
none_button.on_click(filter_none_onclick)
low_button.on_click(filter_low_onclick)
medium_button.on_click(filter_medium_onclick)
high_button.on_click(filter_high_onclick)
max_button.on_click(filter_max_onclick)
save_button.on_click(save_onclick)
remove_filter_button.on_click(remove_filter_onclick)

In [ ]:
image_list = []  #list of image widgets to display all stickers
button_list = []  #List of buttons to select active sticker

#Loops through sticker addresses, createing an image widget and button for each
for i in stickers:
    file = open(i, "rb")  #Open sticker address in a read binary format
    image = file.read()  # read file
    imwig= widgets.Image(  #create image widget
        value=image,
        format='png',
        width=148,  #images are 148x148 pixels
        height=148,
    )
    
    image_list.append(imwig)  #Adds image widget to list
    
    #Creates button to select sticker
    butwig = widgets.Button(
    description='Select',
    disabled=False,
    button_style='success',  #Button is green with up arrow icon
    icon='arrow-up' 
    )
    button_list.append(butwig)  #add button to list
    
    
    
#create slider for sticker scale factor
scaleValue_Slider = widgets.FloatSlider(
    value=1, #slider is set to initatial value of original size
    min=0.1, #Sticker cannot be scaled negatively or by factor of 0 
    max=3.01, #Limits sticker scale factor to avoid oversizing
    step=0.1,  #can scale sticker in steps of 1%
    description='Sticker Scale',
    disabled=False,
    continuous_update=False,  #only updates on mouse release events
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    )
        
#create slider for sticker rotation angle
rotation_Slider = widgets.FloatSlider(
    value=0, #slider is set to initatial value of 0 degrees
    min=-180, 
    max=180, #Sticker rotates between -180 and 180 degrees (full rotation)
    step=1,  #can scale sticker by steps of one degree
    description='Rotation',
    disabled=False,
    continuous_update=False,  #only updates on mouse release events
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    )



#Add click handlers to each button
button_list[0].on_click(sticker0_onclick)
button_list[1].on_click(sticker1_onclick)
button_list[2].on_click(sticker2_onclick)
button_list[3].on_click(sticker3_onclick)

#align widgets in vertical and horizontal boxes 
first_row = HBox(image_list)  #Creates horizontal box containing images
second_row = HBox(button_list)  #Creates horizontal box containing selection buttons
selection_UI= VBox([first_row, second_row])  #Creates vertical box storing horizontal boxes

## Image Cropping and mirroring

The functions below load in the image to be edited. Here it can be mirrored or cropped before passing it to the main image editor. The selected area will be cropped and passed. Clicking the image and dragging will draw a rectangle over the crop area. clicking crop will save the cropped area **Note** the rectangle may take some time to draw

In [ ]:
select_lut = 5

#function called when image clickde
def mouse_click_event(eclick, erelease):  
    global x1,y1,x2,y2
    if mouse_click_event.RS.active:  
        x2 = erelease.xdata  #stores mouse release cooridinates
        y2 = erelease.ydata
        mouse_click_event.RS.set_active(False)
    if not mouse_click_event.RS.active:
        x1 = eclick.xdata
        y1 = eclick.ydata
        mouse_click_event.RS.set_active(True)
        
#creates button to mirror image
mirror_button= widgets.Button(
    description='Mirror',
    disabled=False,
    button_style='warning',
    icon='undo' 
)

#creates button to crop
mirror_button= widgets.Button(
    description='Mirror',
    disabled=False,
    button_style='warning',
    icon='tint' 
)

#creates box hilding buttons
crop_button= widgets.Button(
    description='Crop',
    disabled=False,
    button_style='warning',
    icon='tint' 
)

#adds click handlers to buttons
mirror_button.on_click(mirror_onclick)
crop_button.on_click(crop_onclick)

cropping_ui = HBox([mirror_button,crop_button])  #Creates horizontal box containing selection buttons

fig, ax = plt.subplots() #plot image           

image_path = base_address  #find image
sample_image = Image.open(image_path)  #importimage

plt.imshow(sample_image)  #display

mouse_click_event.RS = RectangleSelector(ax, mouse_click_event,  #adds event hooks to image
                                       drawtype='box',
                                       interactive=True)
plt.connect('button_press_event', mouse_click_event)
plt.connect('button_release_event', mouse_click_event)

#displays buttons
display(cropping_ui)

Displays cropped image and passes it to image editor

## Display Final Image Editing GUI

Here filters and stickers can be applied to the image. To apply filters click the bottoms above the image, here the image can also be save and stickers and filters removed. Stickers can be added by clicking the image. The sticker can be change by clocking the select button below th desired image and scaled or rotated using the slider. Note: changes may take some time to apppear

When applying filters either a blur or sharp can be added along with an LUT filter. The intensity values of the LUT filters do not apply to the negative filter, which has only a single intensity.

Display widgets and plot base image:  **if sliders dont appear please rerun cell**
When the image is saved it is also rotated by the values in the rotation slider, allowning a rotated version of the image to be saved

In [ ]:
display(filter_ui)  #display widgets

#plot image without frame or axes
fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.],frameon=False)
ax.set_axis_off()
fig.add_axes(ax)
plt.imshow(base_image)
cid = fig.canvas.mpl_connect('button_press_event', onclick)  #add event hook to image, activates on click function
    
#display sliders

try:    
    _thread.start_new_thread(slider_Scale, ())
except Exception:
    import traceback
    print(traceback.format_exc())

display(selection_UI)  #display sticker UI below image

Display final image, or saved image if an image has beend saved

In [ ]:
if saved == 0:
    display(base_image)
else:
    display(saved_image)